# Animal Milking Systems (AMS) Trends Analysis 

A jupyter notebook for visualizing and analysing the animal milking systems data provided by Lactanet. 

In [1]:
# -*- coding: utf-8 -*-
"""
AMS_trend_analysis.ipynb
Created on Tues Oct 27th 2020
@author: Amanda
"""

'\nAMS_trend_analysis.ipynb\nCreated on Tues Oct 27th 2020\n@author: Amanda\n'

#### Import the necessary libraries to run the code: 

In [2]:
import pickle
import os 
import pandas as pd
pd.options.mode.chained_assignment = None
import datetime
import matplotlib.pyplot as plt
import matplotlib
# matplotlib.use('Agg')
from matplotlib.ticker import MaxNLocator
import numpy as np
import os
#from readdata import plot_features

#### Load the dataframe into memory.

In [3]:
# Create a list of all the files we will work with
input_path = "./input/anoMilkings_10262020.cpickle"
#input_path = "./input/anoMilkings.cpickle"
result = pickle.load(open(input_path, "rb"))
# Convert all column names to lower case
result = result.rename(str.lower, axis='columns')
result.head()
print("Dataframe shape: ", result.shape)

Dataframe shape:  (83798, 18)


#### Do some cleaning on the dataset:

In [4]:
# Dropping the NA when there is no value on the current columns
print("Before dropping the NA :", result.shape)
result = result.dropna(how="any", subset=["lact_no", "milkng_date"])
print("After dropping the NA :", result.shape)

Before dropping the NA : (83798, 18)
After dropping the NA : (83798, 18)


In [5]:
# Remove Duplicates 
result = result.drop_duplicates(subset=["milkng_date", "milk_wgt", 
                                              "milkng_temp", "milk_flow_avg", 
                                              "milk_flow_max", "fat_pcnt", "prot_pcnt"])

# Print size of the df after dropping duplicates
print("After dropping the duplicates:", result.shape)

After dropping the duplicates: (83798, 18)


In [6]:
# Creating custom columns for MaB, MaBint (integer value), seasons
result["MaB"]= (pd.to_datetime(result.milkng_date) - pd.to_datetime(result.birth_date)).astype('timedelta64[D]')/365.25*12
result["MaBint"] = round(result.MaB) 

# Calculate the season by seperating the year into quarters 
result["WINT"]= result.milkng_date.astype(str).str[5:7].astype(int).isin([1,2,3])
result["SPRI"]= result.milkng_date.astype(str).str[5:7].astype(int).isin([4,5,6])
result["SUMM"]= result.milkng_date.astype(str).str[5:7].astype(int).isin([7,8,9])
result["FALL"]= result.milkng_date.astype(str).str[5:7].astype(int).isin([10,11,12])

#### Add any additional columns we might need:

In [7]:
# Convert all the time columns to datetime objects
# Create a column for the day of milking
result["milkng_day"] = result["milkng_date"].dt.date
result.set_index("milkng_day")

,anm_id,anb_cd,lact_no,lact_start_date,lact_end_date,dim,hrd_id,hrd_prv_cd,birth_date,milkng_date,...,milk_flow_max,fat_pcnt,prot_pcnt,scc,MaB,MaBint,WINT,SPRI,SUMM,FALL
milkng_day,,,,,,,,,,,,,,,,,,,,,
2019-12-12,anm_0,HO,10,2019-12-12,2020-08-31,0,99999,00,2004-02-23,2019-12-12 13:59:55,...,3.4,160.0,382.0,NaN,189.601643,190.0,False,False,False,True
2019-12-12,anm_0,HO,10,2019-12-12,2020-08-31,0,99999,00,2004-02-23,2019-12-12 17:46:26,...,2.2,76.0,571.0,NaN,189.601643,190.0,False,False,False,True
2019-12-13,anm_0,HO,10,2019-12-12,2020-08-31,1,99999,00,2004-02-23,2019-12-13 05:27:20,...,2.6,327.0,518.0,NaN,189.634497,190.0,False,False,False,True
2019-12-13,anm_0,HO,10,2019-12-12,2020-08-31,1,99999,00,2004-02-23,2019-12-13 17:04:23,...,3.4,417.0,474.0,NaN,189.634497,190.0,False,False,False,True
2019-12-14,anm_0,HO,10,2019-12-12,2020-08-31,2,99999,00,2004-02-23,2019-12-14 05:45:58,...,3.6,385.0,442.0,NaN,189.667351,190.0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-25,anm_120,HO,1,2020-08-13,NaT,73,99999,00,2018-11-05,2020-10-25 01:22:46,...,3.3,329.0,331.0,NaN,23.655031,24.0,False,False,False,True
2020-10-25,anm_120,HO,1,2020-08-13,NaT,73,99999,00,2018-11-05,2020-10-25 08:10:24,...,3.5,380.0,329.0,18.0,23.655031,24.0,False,False,False,True
2020-10-25,anm_120,HO,1,2020-08-13,NaT,73,99999,00,2018-11-05,2020-10-25 14:36:44,...,3.4,530.0,321.0,NaN,23.655031,24.0,False,False,False,True


#### Group the data by animal, laction #, milking day, week and month:

In [8]:
print("Grouping Data....")
anm_group = result.groupby(["anm_id"])
anm_group_lactation = result.groupby(["anm_id", pd.Grouper(key="lact_no")])
sample1 = pd.DataFrame(dtype=float)

# can use freq = "M", "W" with the key variable to group by month and week 
#anm_group = result.groupby(["anm_ident", pd.Grouper(key="milkng_day")]).milk_wgt.agg(['count','min','max','mean'])
anm_group_day = result.groupby(["anm_id", pd.Grouper(key="milkng_day")])
anm_group_day_summ = result.groupby(["anm_id", pd.Grouper(key="milkng_day")]).agg({"milkng_date":['count'],
                                                                                 "milk_wgt": ['sum','min','max','mean', 'std', 'var', 'sem'], 
                                                                                 "milkng_temp" : ['min','max','mean', 'std', 'var', 'sem'],
                                                                                 "milk_flow_avg": ['min','max','mean', 'std', 'var', 'sem'],
                                                                                 "milk_flow_max": ['min','max','mean', 'std', 'var', 'sem'],
                                                                                 "fat_pcnt": ['min','max','mean', 'std', 'var', 'sem'],
                                                                                 "prot_pcnt": ['min','max','mean', 'std', 'var', 'sem'],
                                                                                 "scc": ['min','max','mean', 'std', 'var', 'sem']
                                                                                  })
print("Finished. Number of day groups created: ", len(anm_group_day.groups))

print("Number of animal groups created: ",  len(anm_group.groups))

Grouping Data....
Finished. Number of day groups created:  27592
Number of animal groups created:  121


#### Loop through the groups and computer the rolling mean for milk yield and time difference. 

In [9]:
def get_24hr_yield(grouper, name):
    df = grouper.get_group(name)
    df.loc[:,"milk_time_diff"] = pd.to_timedelta(df["milkng_date"].diff())
    df["milk_time_diff"] = (df["milk_time_diff"].dt.total_seconds()/3600).map('{:,.4f}'.format)
    df["milk_time_diff"] = df["milk_time_diff"].str.replace(',','').astype(float)
    df.loc[:,"rolling_time"] = df["milk_time_diff"].rolling(window=13).mean()
    df.loc[:,"rolling_yield"] = df["milk_wgt"].rolling(window=13).mean()
    df.loc[:, "24_hr_yield"] = (df["rolling_yield"]/df["rolling_time"])*24

    return df

### Turn into a function that calculates 24 prot pcnt, log scc, 


In [10]:
for name, group in anm_group:
    #print("Name: ", name)
    #ax1 = plt.figure().gca()
    df = get_24hr_yield(anm_group, name)
    
    # Here this function only works for one single animal, will need to find a way to modify ALL existing animal groups with
    # this function. 

In [11]:
# Create a sample for viewing using ANM 92 
df = get_24hr_yield(anm_group, "anm_92")
df.shape

(1027, 29)

In [12]:
# Print some info about the dataframe to screen 
print("ANM_92 data: ", df.head())
print("ANM_92 fields: ", df.columns)
df.describe()
#df["milk_wgt"].describe()

ANM_92 data:         anm_id anb_cd  lact_no lact_start_date lact_end_date  dim  hrd_id  \
68226  anm_92     HO        1      2019-08-18    2020-05-27  107   99999   
68227  anm_92     HO        1      2019-08-18    2020-05-27  107   99999   
68228  anm_92     HO        1      2019-08-18    2020-05-27  107   99999   
68229  anm_92     HO        1      2019-08-18    2020-05-27  107   99999   
68230  anm_92     HO        1      2019-08-18    2020-05-27  108   99999   

      hrd_prv_cd birth_date         milkng_date  ...  MaBint   WINT   SPRI  \
68226         00 2017-10-06 2019-12-03 05:51:12  ...    26.0  False  False   
68227         00 2017-10-06 2019-12-03 11:57:10  ...    26.0  False  False   
68228         00 2017-10-06 2019-12-03 17:57:54  ...    26.0  False  False   
68229         00 2017-10-06 2019-12-03 23:24:32  ...    26.0  False  False   
68230         00 2017-10-06 2019-12-04 07:17:06  ...    26.0  False  False   

        SUMM  FALL  milkng_day  milk_time_diff  rolling_time

,lact_no,dim,hrd_id,milk_wgt,milkng_code,milkng_temp,milk_flow_avg,milk_flow_max,fat_pcnt,prot_pcnt,scc,MaB,MaBint,milk_time_diff,rolling_time,rolling_yield,24_hr_yield
count,1027.000000,1027.000000,1027.0,1027.000000,1027.000000,1026.000000,1025.000000,1027.000000,1026.000000,1026.000000,328.000000,1027.000000,1027.000000,1026.000000,1014.000000,1015.000000,1014.000000
mean,1.483934,126.739046,99999.0,8.890847,0.006816,38.281287,2.427805,3.518598,439.040936,315.254386,79.756098,31.768485,31.752678,7.670875,7.696726,8.903895,35.158619
std,0.499985,83.709630,0.0,1.867819,0.082317,1.369259,0.372016,0.424189,48.211508,22.910315,120.350615,3.255474,3.289356,37.462229,10.745306,1.209268,7.937320
min,1.000000,0.000000,99999.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,25.889117,26.000000,0.120600,4.050762,2.638462,0.697955
25%,1.000000,51.000000,99999.0,8.200000,0.000000,38.000000,2.200000,3.300000,422.000000,303.000000,42.000000,28.845996,29.000000,4.978575,5.137800,8.773077,31.668031
50%,1.000000,111.000000,99999.0,8.800000,0.000000,38.300000,2.400000,3.500000,443.000000,312.000000,58.000000,31.375770,31.000000,6.102650,6.417727,9.130769,33.645225
75%,2.000000,202.000000,99999.0,9.800000,0.000000,38.600000,2.600000,3.700000,461.000000,323.750000,83.000000,34.759754,35.000000,6.704200,6.804019,9.469231,42.534409
max,2.000000,282.000000,99999.0,15.700000,1.000000,40.700000,3.500000,7.700000,626.000000,457.000000,1865.000000,36.665298,37.000000,1112.945000,96.720862,11.961538,46.843659


In [13]:
import chart_studio.plotly as py
import plotly.graph_objs as go # import plotly graph objects
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots

# Jupyter Setup 
init_notebook_mode(connected=True) # allows us to visualize the code in a jupyter notebook

In [14]:
# Create a plotly figure window
fig = go.Figure()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(go.Scatter(x=anm_group_day_summ.loc["anm_92"][('milkng_date', 'count')].index, y=anm_group_day_summ.loc["anm_99"]["milk_wgt"]["sum"],
                    mode='lines',
                    name='Daily Milk Weight',
                    hovertemplate='<b>Date: %{x}</b><br> Milk yield: %{y:.2f}<br>'))

fig.add_trace(go.Scatter(x=df['milkng_date'], y=df["24_hr_yield"],
                    mode='lines',
                    name='Estimated 24-Hour Yield',
                    hovertemplate='<b>Date: %{x}</b><br> Estimated milk yield: %{y:.2f}<br>'))

fig.add_trace(go.Scatter(x=df['milkng_date'], y=df["dim"],
                    mode='lines',
                    name='Days in Milk',
                    customdata = df['lact_no'],
                    hovertemplate='<b>Date: %{x}</b><br>Days in Milk: %{y}<br> Lactation number: %{customdata}'),
                    secondary_y=True)


# Set x-axis title
#fig.update_xaxes(range =["2019-11-01", "2020-04-30"], title_text="<b>Date<b>")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Milk Weight (Kg)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Days in Milk (Count)</b>", secondary_y=True)

fig.update_layout(title={'text': "Milk Yield for ANM_92", 'x' :0.4, 'y':0.9, 'xanchor': 'center', 'yanchor': 'top'})

fig.show()


In [15]:
"""
print(anm_group_day_summ.loc["anm_99"]["milk_wgt"]["sum"])
print(df["24_hr_yield"])
print(df["rolling_yield"])

anm_group_day_summ.loc["anm_99"].columns
anm_group_day_summ.loc["anm_99"][('milkng_date', 'count')].index
"""

'\nprint(anm_group_day_summ.loc["anm_99"]["milk_wgt"]["sum"])\nprint(df["24_hr_yield"])\nprint(df["rolling_yield"])\n\nanm_group_day_summ.loc["anm_99"].columns\nanm_group_day_summ.loc["anm_99"][(\'milkng_date\', \'count\')].index\n'

In [16]:
"""
import plotly.express as px

fig = px.line(df, x='milkng_date', y=["24_hr_yield", "milk_wgt", 'dim'], hover_data=['lact_no'])
fig.show()
""""""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-16-54781d055c81>, line 6)